For this Notebook to work correctly, the virtual memory on the Windows 10 machine where this Notebook is being run must be increased. This is because the notebook will download and process satellite images that are, generally, very large. In order to achieve this, there are some system modifications that need to be done before running the Notebook. 
To increase the virtual memory of the machine: 

Go to Control panel---> System---> Advanced System Settings. 

In the window that appears select the Advance tab, then settings, in the new window select the advanced tab, the click on change. Here, untick the "Automatically manage paging file for all drives" Select then the Custom size and in the initial size (MB) box. Enter 4096 and in the Maximum size (MB) box enter 7000 or 8000. (The maximum size can be increased if the mosaic to be created is very large).

In [ ]:
#Importing the neccesary libraries for downloading data 
import requests
import time
from datetime import datetime
import pandas as pd
from pathlib import Path
import xmltodict
from zipfile import ZipFile

The following cell will create a folder, where the notebook is located, and all
the downloaded images will be stored there 

In [ ]:
# get input for "test name" eg 'PRD_STANDARDISED_TEST_WPS_S1x10_RUN1'
RUNNAME = input()

In [ ]:
output_fmt='%Y%m%dT%H%M%S'
pretty_fmt='%Y-%m-%d %H:%M:%S'
headers = {'Content-type': 'application/xml','User-Agent': 'curl/7.65.1'}

wps_test_config = {
        'gs:Download':{
            'template_xml':'gsdownload-template.xml',
            'mimetype':'application/zip',
            'output':'result.zip',
        }}

In [ ]:
# Load list of granules from dowloaded csv 
#(Those are the resulting csv from eods-api-example-generate-cloudless-mosaic Notebook)
col_list = ["typename"]
min_cloud_per_granule = pd.read_csv(r"C:\Users\.......\min_cloud_per_granule.csv")

min_cloud_per_granule_per_orbit = pd.read_csv(r"C:\Users\.........\min-cloud-per-granule-per-orbit.csv")

In [ ]:
### USER INPUT
### Enter your token in PRD_DM = 

wps_job_request_list = [
     {'tool':'gs:Download','layer_list':min_cloud_per_granule_per_orbit["typename"],'dl_bool':True}
]

PRD_DM = "Your token goes here"
ACCESS_TOKEN = PRD_DM

ENV_PRD = 'https://earthobs.defra.gov.uk'
wps_env = ENV_PRD

wps_server = wps_env + '/geoserver/ows/?access_token=' + ACCESS_TOKEN + '&SERVICE=WPS&VERSION=1.0.0'

In [ ]:
# submitting the WPS request(s)

def mod_the_xml(xml_name,layer_name):
    """
    function read xml payload template and modify the payload with the config
    """
    
    with open(Path.cwd() / 'xml' / xml_name,'r') as template_xml:
        file_data = template_xml.read()
    modded_xml = file_data.replace('template_layer_name', layer_name)
    
    return modded_xml

def submit_wps_request(payload):
    """
    function submit the wps POST request
    """

    try:
        wps_submit_response = requests.post(wps_server + '&REQUEST=EXECUTE', data=payload, headers=headers, verify=True)
        print('\n' + datetime.utcnow().isoformat() + ' :: request POST url =' + wps_server + '&REQUEST=EXECUTE')
    except:
        print('wps POST was not successful')
    
    # receive status code
    timestamp_submission = datetime.utcnow()
    status_code = wps_submit_response.status_code
        
    if status_code == 200 and not wps_submit_response.text.find('ExceptionReport') > 0:
        execution_id = wps_submit_response.text.split('executionId=')[1].split('&')[0]        
        print('\n' + datetime.utcnow().isoformat() + ':: WPS job ' + execution_id + ' was successfully submitted')
        print('\n' + datetime.utcnow().isoformat() + ':: ' + str(wps_submit_response.content))
        return execution_id
    else:
        print(datetime.utcnow().isoformat() + ' :: wps request was not successful :: STATUS = ' + str(status_code))
        print(datetime.utcnow().isoformat() + ' :: response = ' + str(wps_submit_response.content))
        return None

execution_id_dict={}

for item in wps_job_request_list:
        
    for lyr in item['layer_list']:
        print('\n' + datetime.utcnow().isoformat() + ' :: tool=' + item['tool'] + ', lyr=' + lyr + ', download=' + str(item['dl_bool']))
        modded_xml = mod_the_xml(wps_test_config[item['tool']]['template_xml'],lyr)
        execution_id = submit_wps_request(modded_xml)
        execution_id_dict.update({execution_id:lyr})
        submission_time = datetime.utcnow()

print('\n' + datetime.utcnow().isoformat() + ' :: A TOTAL OF ' + str(len(execution_id_dict)) +  ' WPS REQUESTS SUBMITTED')

print('\n' + datetime.utcnow().isoformat() + ' :: The ExecutionIDs are :: ' + str(execution_id_dict.keys()))

print('\n' + datetime.utcnow().isoformat() + ' :: WPS Check Executions URL is ' + wps_server + '&REQUEST=GetExecutions')

In [ ]:
# poll the status page and wait until all jobs are completed

# create output directory and log file
Path(Path.cwd() / 'output').mkdir(parents=True, exist_ok=True)
my_path = Path.cwd() / 'output' / str(submission_time.strftime(output_fmt) + '-' + RUNNAME)
my_path.mkdir(parents=True, exist_ok=True)
log_file_name = my_path / str(submission_time.strftime(output_fmt) + '-' + RUNNAME + '-wps-log.csv')
poll_frequency_sec = 60
frames = []

# poll api for 1 hour unless all jobs submitted are NOT STATUS=RUNNING
for i in range(1,3600):
    
    check_time = datetime.utcnow()
    if i == 1:
        print('\n#' + format(i, '03') + ' of #n   :: ' + submission_time.isoformat() + ' :: Logging file: ' + str(log_file_name))
    
    print('\n#' + format(i, '03') + ' of #n   :: ' + check_time.isoformat() + ' :: Poll Time')
    
    start_index = 0
    matching_of_dicts = []
    for page_counter in range(0,1000,10):

        # constrcut the paging URL and make the request
        all_status_url = wps_server + '&REQUEST=GetExecutions&startIndex=' + str(page_counter)

        r = requests.get(all_status_url,headers=headers)

        # parse the xml to an ordered dict using 3rd party imported module xmltodict
        d = xmltodict.parse(r.content)

        # grab the dicts of 'wps:ExecuteResponse' and add to a list
        jobs_list = [value for value in d['wps:GetExecutionsResponse']['wps:ExecuteResponse']]

        # add this to a master list
        matching_of_dicts.append(jobs_list)

        # if there is no 'next' attribute, then you're on the last xml page so break out the loop
        if not any('@next' == key for key in list(d['wps:GetExecutionsResponse'].keys())):
            break

    num_of_xml_pages = int(page_counter / 10) + 1
    duration_to_parse_xml_dt = datetime.utcnow() - check_time
    duration_to_parse_xml_sec = round(duration_to_parse_xml_dt.total_seconds(),0)
    print('#' + format(i, '03') + ' of #n   :: ' + datetime.utcnow().isoformat() + ' :: ' + str(num_of_xml_pages) + ' xml pages parsed in ' + str(duration_to_parse_xml_sec) + ' seconds')
            
    # now flatten the list into all responses
    response_list = [item['wps:Status'] for sublist in matching_of_dicts for item in sublist]

    # parse dict to pandas dataframe and set the index
    df_temp = pd.DataFrame.from_dict(response_list)
    df = df_temp.set_index('wps:JobID')

    # filter df on current job_ids
    filter_df  = df[df.index.isin(execution_id_dict.keys())]
            
    # add "check time" as column
    filter_df.loc[:,'check_time'] = check_time.isoformat()
    
    # append on some extra info to the dataframe using the index
    for index, row in filter_df.iterrows():
        filter_df.loc[index,'layer_name'] = execution_id_dict[index]
        filter_df.loc[index,'dl_url'] = wps_server + '&REQUEST=GetExecutionResult&EXECUTIONID='  + index + '&outputId=' + wps_test_config['gs:Download']['output'] + '&mimetype=' + wps_test_config['gs:Download']['mimetype']
    
    # sort out the indices and concatenate dataframes together and output to csv
    filter_df.reset_index(inplace=True)
    filter_df.set_index(['wps:JobID','check_time'],inplace=True)
    frames.append(filter_df)
    rolling_merged_df = pd.concat(frames)
    rolling_merged_df.to_csv(log_file_name)
        
    # if NO running processes, then break, otherwise loop again
    if not any(filter_df['wps:Status'] == 'RUNNING'):
        break
    print('#' + format(i, '03') + ' of #n   :: ' + datetime.utcnow().isoformat() + ' :: Jobs are still "STATUS=RUNNING"...script will poll again in ' + str(poll_frequency_sec) + 'sec time ...')
    
    time.sleep(poll_frequency_sec)
    
print('\n#' + format(i, '03') + ' of #' + format(i, '03') + ' :: ' + datetime.utcnow().isoformat() + ' :: Script has finished')

In [ ]:
# export summary wps jobs with "runtimes"
sumdf = rolling_merged_df.reset_index()
grouped = sumdf.groupby(['wps:JobID','wps:Status']).first()
grouped.to_csv(my_path / str(submission_time.strftime(output_fmt) + '-' + RUNNAME + '-summary-wps-log.csv'))

In [ ]:
# DOWNLOAD, WRITE FILE AND RENAME FILE
for index, row in filter_df.iterrows():
    if row['wps:Status'] == 'SUCCEEDED':
        print('\n\n' + datetime.utcnow().isoformat() + ' :: DOWNLOAD START :: URL = ' + row['dl_url'])        
        response = requests.get(row['dl_url'],headers=headers);
        print(datetime.utcnow().isoformat() + ' :: DOWNLOAD COMPLETE')
        local_file_name = my_path / str(row['layer_name'].split(':')[-1] + '.zip')
        print(datetime.utcnow().isoformat() + ' :: FILE WRITE START :: FILE = ' + str(local_file_name))        
        with open(local_file_name, 'wb') as f:
            f.write(response.content);
            f.close();        
        print(datetime.utcnow().isoformat() + ' :: FILE WRITE COMPLETE')
        print(datetime.utcnow().isoformat() + ' :: EXTRACT ARCHIVE FILE STARTED')
                
        # TODO. Check if downloaded file is actually a zip file type. Handle exception for an xml error report?
        zip_ref = ZipFile(local_file_name, 'r')
        zip_ref.extractall(my_path)
        zip_ref.close()
        print(datetime.utcnow().isoformat() + ' :: EXTRACT ARCHIVE FILE COMPLETE')
        print(datetime.utcnow().isoformat() + ' :: RENAME/CLEAN UP DIRECTORY STARTED')
        for f in zip_ref.filelist:
            if f.filename.endswith(".sld"):
                Path(my_path / f.filename).unlink()
            
            # rename the uuid.tiff to layer_name.tif
            elif f.filename.endswith(".tif") or f.filename.endswith(".tiff"):
                Path(my_path / f.filename).rename(my_path / str(row['layer_name'].split(':')[-1] + '.tif'))
            else:
                raise ValueError("ERROR - some unknown file has been given:", f)
        
        # remove the zip file from disk
        Path(local_file_name).unlink()
        print(datetime.utcnow().isoformat() + ' :: RENAME/CLEAN UP DIRECTORY COMPLETE')

print('\n\n' + datetime.utcnow().isoformat() + ' :: #### SCRIPT FINISHED')

In [ ]:
#Importing libraries for Mosaic creation
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import rsgislib
from rsgislib import imageutils
import glob
import os
import os.path
import sys

In [ ]:
#Setting up the folder and the tif files for the mosaic
dirpath= my_path
search_critiria = "*.tif"
q = os.path.join(dirpath,search_critiria)
S2_fps = glob.glob(q)

In [ ]:
##Importing libraries for converting .tif to .kea for the Mosaic creation
import os.path
import sys

In [ ]:
#Setting up the convertion 
def replaceGTIFF_kea(inputtext):
    outputtext = ""    
    for w in inputtext:
        w = w.replace("GEOTIFF","KEA")
        w = w.replace(".TIF",".kea")
        # this line should be unnecessary since Landsat MTL files use capital letters       
        # but just in case you have one that doesn't
        w = w.replace(".tif",".kea")
        outputtext += w
    return outputtex

In [ ]:
# find all *.TIF files and *MTL.txt files in the current directory
directory = os.chdir(my_path)
dirFileList = os.listdir(directory)

In [ ]:
# print dirFileList
tifFileList = [f for f in dirFileList if ((f[-4:]=='.TIF')or(f[-4:]=='.tif'))]
MTLFileList = [f for f in dirFileList if (f[-7:]=='MTL.txt')]

In [ ]:
#output format (GDAL code)
outFormat = 'KEA'

In [ ]:
# run gdal_translate on all TIFs to convert to KEA
for t in tifFileList:
    gdaltranscmd = "gdal_translate -of "+outFormat+" "+t+" "+t[:-4]+".kea"
    print (gdaltranscmd)
    os.system(gdaltranscmd)

In [ ]:
# create a new header file referring to .kea files rather than .TIF
for m in MTLFileList:
    inputtext = file(m).readlines()
    outputtext = replaceGTIFF_kea(inputtext)
    outputfilebase = m[:-4]
    outputfile = outputfilebase + "_kea.txt"
    out = file(outputfile,"w")
    out.write(outputtext)
    out.close()

In [ ]:
#Setting up the paths for finding the created .kea files
dirpath= my_path
search_critiria_kea = "*.kea"
q = os.path.join(dirpath,search_critiria_kea)
S2 = glob.glob(q)

In [ ]:
S2

In [ ]:
# Mosaicing all the .kea files 
inputList = S2
outImage = r'C:/Users/......../Mosaic.kea'
backgroundVal = 0.
skipVal = 0.
skipBand = 1
overlapBehaviour = 0
gdalformat = 'KEA'
datatype = rsgislib.TYPE_32FLOAT
imageutils.createImageMosaic(inputList, outImage, backgroundVal, skipVal, skipBand, overlapBehaviour, gdalformat, datatype)

In [ ]:
#Converting the created .kea mosaic back to .tif 
def replaceGTIFF_kea(inputtext):
    outputtext = ""    
    for w in inputtext:
        w = w.replace("KEA","GEOTIFF")
        w = w.replace(".kea",".TIF")
        # this line should be unnecessary since Landsat MTL files use capital letters       
        # but just in case you have one that doesn't
        w = w.replace(".kea",".tif")
        outputtext += w
    return outputtex

In [ ]:
# find all *.kea files and *MTL.txt files in the current directory
directory = os.chdir()
dirFileList = os.listdir(directory)

In [ ]:
# print dirFileList
tifFileList = [f for f in dirFileList if ((f[-4:]=='.KEA')or(f[-4:]=='.kea'))]
MTLFileList = [f for f in dirFileList if (f[-7:]=='MTL.txt')]

In [ ]:
#output format (GDAL code)
outFormat = 'GTiff'

In [ ]:
#run gdal_translate on the .KEA Mosaic  to convert to .tiff
for t in tifFileList:
    gdaltranscmd = "gdal_translate -of "+outFormat+" "+t+" "+t[:-4]+".tif"
    print (gdaltranscmd)
    os.system(gdaltranscmd)

In [ ]:
#Importing libraries for clipping 
import fiona
import rasterio 
import rasterio.mask
#Importing the shapefile that will be used for clipping
with fiona.open(r"C:\Users\.....\your_shapefile_for_cliiping.shp") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]



In [ ]:
#Clipping the Mosaic with the shapefile
with rasterio.open(r"C:\Users\..............\Mosaic.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [ ]:
#Saving the created cliped raster
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(r"C:\Users\.................\County.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
#Importing libraires for NDVI
import numpy as np
from rasterio.plot import show
import earthpy.spatial as es

In [ ]:
#Importing the clipped Mosaic
Sentinel_data_path = r"C:\Users\................\County.tif"

with rasterio.open(Sentinel_data_path) as src:
    Sentinel_data = src.read()

# View shape of the data
Sentinel_data.shape

In [ ]:
#Calculate the NDVI for the clipped Mosaic. 
Sentinel_ndvi = es.normalized_diff(Sentinel_data[7], Sentinel_data[3])

In [ ]:
#Importing libraries for plotting the results.
import earthpy.plot as ep
import matplotlib.pyplot as plt

In [ ]:
#Ploting the created NDVI
ep.plot_bands(Sentinel_ndvi, 
              cmap='PiYG',
              scale=False,
              vmin=-1, vmax=1,
              title="Sentinel 2 NDVI")
plt.show()

In [ ]:
#Creating a histogram showing the attribute distribution of the NDVI
ep.hist(Sentinel_ndvi,
        figsize=(12, 6),
        title=["NDVI: Distribution of pixels\Sentinel 2"])

plt.show()

In [ ]:
#Viewing the metadata of the NDVI
with rasterio.open(Sentinel_data_path) as src:
    Sentinel_data = src.read()
    Sentinel_meta = src.profile

Sentinel_meta

In [ ]:
#Saving the CRS for the NDVI
Sentinel_transform = Sentinel_meta["transform"]
Sentinel_crs = Sentinel_meta["crs"]

# View spatial attributes
Sentinel_transform, Sentinel_crs

In [ ]:
type(Sentinel_ndvi), Sentinel_ndvi.dtype

In [ ]:
Sentinel_meta

In [ ]:
# Change the count or number of bands from 4 to 1
Sentinel_meta['count'] = 1

In [ ]:
#Saving the created NDVI raster
Sentinel_ndvi_outpath = r"C:\Users\...........\NDVI.tif"


with rasterio.open(Sentinel_ndvi_outpath, 'w', **Sentinel_meta) as dst:
    dst.write(Sentinel_ndvi, 1)